In [ ]:
import pandas as pd 
from pandas import DataFrame
import nltk

df = pd.read_csv('Data/InterviewDataResponses.csv')

In [ ]:
df.columns =  [
'timestamp','howLearnComp',
'negAspectComp','dataLab','magicWand','csCourseFuture','currFactor','changeThoughtBerkeley','changePerceptnCS',
'funAspectClass','unfunAspectClass','respectCourseStaff','courseCulture61A','changeThoughtBerkeley2',
'moreCSClasses','perceptStudyCSChange','feelMinorityCS','whatFirstBroughtCourseAttention','reservationTakingClass',
'probStudyCS','relationshipProgrammerScientist','comptThink','enjoyCourse','enjoyProbSolv',
'priorCSbeforeClass','academicStrenghts','otherAcademicInterest','majorBerkeley','answerDiff','strengthCS',
'coolestAspectCS','thinkSomeOneCSDo','benefitStudyCS','theName','gender','priorExposureToCS','cs10','cs61A',
'race','acadClass','experienceSnap','anythingLikeToShare','extra','expectGetOutOfCourse',
'q_CT','q2','q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16','q17']


In [ ]:
cs10 = df[df.cs10 == 'yes']
cs61a = df[df.cs61A == 'yes']
cs10.reset_index(drop=True)
cs61a.reset_index(drop=True)

cs10_female = cs10[cs10.gender == 'female' ]
cs10_male = cs10[cs10.gender == 'male']
cs10_female = cs10_female.reset_index(drop=True)
cs10_male = cs10_male.reset_index(drop=True)


cs61a_female = cs61a[cs61a.gender == 'female']
cs61a_female = cs61a_female[pd.isnull(cs61a_female.cs10)]

cs61a_male = cs61a[cs61a.gender == 'male']
cs61a_male = cs61a_male[pd.isnull(cs61a_male.cs10)]
cs61a_female = cs61a_female.reset_index(drop=True)
cs61a_male = cs61a_male.reset_index(drop=True)


In [ ]:
from nltk.corpus import stopwords

stopwords = stopwords.words('english')


def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.node=='NP'):
        yield subtree.leaves()

def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    #word = stemmer.stem_word(word)
    word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


def get_terms(tree):
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term

        # Used when tokenizing words
sentence_re = r'''(?x)      # set flag to allow verbose regexps
      ([A-Z])(\.[A-Z])+\.?  # abbreviations, e.g. U.S.A.
    | \w+(-\w+)*            # words with optional internal hyphens
    | \$?\d+(\.\d+)?%?      # currency and percentages, e.g. $12.40, 82%
    | \.\.\.                # ellipsis
    | [][.,;"'?():-_`]      # these are separate tokens
'''

lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

#Taken from Su Nam Kim Paper...
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """

In [ ]:

def iterateCodes(dfName, category, item):
    
    
        dfCol = []
        dfCol.append(dfName+category+'.columns')
        
        code_tmp = {}
        
        dfItem = []
        dfItem.append(dfName+category+'.iterrows()')
        for row, columns in eval(dfItem[0]):
            if pd.isnull(columns[item]):
                continue
            else:
                text = str(columns[item]).split()
                text = ' '.join(text)
                chunker = nltk.RegexpParser(grammar)
                toks = nltk.regexp_tokenize(text, sentence_re)
                postoks = nltk.tag.pos_tag(toks)
                tree = chunker.parse(postoks)
                terms = get_terms(tree)
                    
                code = []
                for term in terms:
                    for word in term: 
                        if code_tmp.has_key(word):
                            code_tmp[word] += 1
                        else:
                            code_tmp[word] = 0 
                    
        return code_tmp
    

def printCode(coding):
    
    window = 10

    for key in sorted(coding):
        if coding[key] > 1 and (window > 0 and window <=10):
            print "%s" % (key),
            window = window - 1
        elif window < 1:
            print '\n'
            window = 10
    print '\n------ E N D-------\n\n'

In [ ]:
items = cs10.columns.values
for item in items:
    coding = {}
    coding = iterateCodes('cs10', '', item)
    print item, "CS10\n"
    printCode(coding)

    coding = iterateCodes('cs61a', '', item)
    print item, "CS61A\n"
    printCode(coding)
